# Image Classification With A Deep Convolutional Neural Network (AlexNet)
This notebook details the steps taken to implement and train a Deep Neural Network(DNN) on the [Cifar-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset in order to perform the computer vision task of image classification on the Cifar-10 test dataset partition. 

To build, train and test a neural network, the libraries TensorFlow and Keras are leveraged to provide the features and methods required to build the components of a neural network. Supporting libraries such as Matplotlib and Numpy are utlised for visualisation and data processing

**[TensorFlow](https://www.tensorflow.org/)**: An open-source platform for implementing, training, and deploying machine learning models.

**[Keras](https://keras.io/)**: An open-source library used to implement neural network architectures that run on both CPUs and GPUs.

**[Matplotlib](https://matplotlib.org/)**: Tool utilized to create visualization plots in Python such as charts, graphs and more

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

## Cifar-10 Dataset

The [Cifar-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset consists of images of animals and everyday vehicles.
The dataset was put together by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton. More specifically, it contains 50,000 training examples and 10,000 testing examples, that are all color images with the dimension 32 x 32 categorized into 10 classes.

[**Keras Dataset**](https://keras.io/api/datasets/) (keras.datasets)

In the practical world of machine learning, obtaining dataset is costly in terms of time, money and effort.
But a few introductory datasets have been collated together to help students and researchers build and analyse the performance of models on specifics datasets.
Some of these dataset reside within the Keras API and are easily accessible through the [load_data()](https://keras.io/api/datasets/cifar10/#loaddata-function) method of each respective dataset.

To load the Cifar-10 dataset: `keras.datasets.cifar10.load_data()`

**Classes**
| Label | Description |
|-------|-------------|
| 0     | airplane    |
| 1     | automobile  |
| 2     | bird        |
| 3     | cat         |
| 4     | deer        |
| 5     | dog         |
| 6     | frog        |
| 7     | horse       |
| 8     | ship        |
| 9     | truck       |

**Dataset Partitions**

For this particular classification task, 45,000 training images, 10,000 test images, and 5,000 validation images are utilized.
- Training Data: This is the group of our dataset used to train the neural network directly. Training data refers to the dataset partition exposed to the neural network during training.
- Validation Data: This group of the dataset is utilized during training to assess the performance of the network at various iterations.
- Test Data: This partition of the dataset evaluates the performance of our network after the completion of the training phase.

**The code snippet below unpacks the tuple of numpy arrays that correspons to the training images and labels, as well as the testing images and labels**

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

In [ ]:
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### Validation Data
The validation partition of the dataset is obtained from taking a slice of the training data. More specifically, the first 5000 training images and labels are assigned to the validation data.
The new training data is reassigned to every training data element but the first 5000 images in the original training data.

In [ ]:
validation_images, validation_labels = train_images[:5000], train_labels[:5000]
train_images, train_labels = train_images[5000:], train_labels[5000:]

## Input Pipeline and TensorFlow Dataset

Input pipelines are used as a method of streaming data to a model in an automated fashion. Input pipelines can be made responsible for the automation of preprocessing steps taken on input data before presented to the network during training.

In this notebook we create input pipelines for our training, testing and validation data using the [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) API. With well constructed pipelines transformations and augmentation can be made to data points or batches of the dataset partitions in an memory efficient manner.

The source of the data used to create the Dataset Object for our input pipeline is the numpy arrays of the dataset partitions created in the cells above. More specfically the `tf.data.Dataset.from_tensor_slices()` allows for the creation of the required Dataset object.

Below we create the Dataset Objects `train_da`, `test_ds` and `validation_ds` which represent the input pipelines for the training, testing and validation data respectively.

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

## Visualising Training Data

The Dataset object returned tf.data.Dataset can be iterated over, so we can view a portion of the dataset using pyplot.

Below is a visualisation of the first five images and corresponding labels.

In [ ]:
plt.figure(figsize=(20,20))
for i, (image, label) in enumerate(train_ds.take(5)):
    ax = plt.subplot(5,5,i+1)
    plt.imshow(image)
    plt.title(CLASS_NAMES[label.numpy()[0]])
    plt.axis('off')

## Image Preproccessing

**Standardization**
The TensorFlow module [tf.image](https://www.tensorflow.org/api_docs/python/tf/image) provides a suite of processing and transformation functions that can be applied on image dataset, so resizing, recoloring, cropping and other common transformation become trivial. 

In this notebook the images are resized from the original dimensions of 32x32 to 227x227, this is because the input layer of the AlexNet architecture expects images of size 227x227. In addition to resizing, the images are normalized to have a mean of 0 and standard deviation of 1

In [ ]:
def image_preprocessing(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

In [ ]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

### Finializing the Input pipelines



In [ ]:
train_ds = (train_ds
                  .map(image_preprocessing)
                  .cache()
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

test_ds = (test_ds
                  .map(image_preprocessing)
                  .cache()
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

validation_ds = (validation_ds
                  .map(image_preprocessing)
                  .cache()
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_ds,
          epochs=30,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

In [ ]:
model.evaluate(test_ds)

In [ ]:
# AlexNet with Augmentation layers
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.RandomRotation(0.2, input_shape=(227,227,3)),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
    keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.01, momentum=0.9), metrics=['accuracy'])

model.fit(train_ds,
          epochs=400,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

model.evaluate(test_ds)